# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [75]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = None, names = columns))


In [76]:
# check if data has been loaded
#fires_dt = pd.read_csv('../../05_src/data/fires/forestfires.csv', header=0, names=columns)

# Inspect the dataset
#print(fires_dt.head())
#print(fires_dt.info())

# Get X and Y

Create the features data frame and target data.

In [77]:

target = 'area'
X = fires_dt.drop(columns=[target])
y = fires_dt[target]


print(X.head())
print(y.head())

  coord_x coord_y  month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain
0       X       Y  month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain
1       7       5    mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7     0
2       7       4    oct  tue  90.6  35.4  669.1  6.7    18  33   0.9     0
3       7       4    oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3     0
4       8       6    mar  fri  91.7  33.3   77.5    9   8.3  97     4   0.2
0    area
1       0
2       0
3       0
4       0
Name: area, dtype: object


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [78]:
# preproc1
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.pipeline import Pipeline

# numeric and categorical features
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [79]:
# preproc2
preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('scaler', StandardScaler()),
            ('power', PowerTransformer())
        ]), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
import numpy as np


In [81]:

baseline_regressor = LinearRegression()
advanced_regressor = RandomForestRegressor(random_state=42)

# Pipeline A: preproc1 + baseline (linear regression)
pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', baseline_regressor)
])

In [82]:
# Pipeline B = preproc2 + baseline (linear regression)
pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', baseline_regressor)
])

In [83]:
# Pipeline C = preproc1 + advanced model(random forest)
pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', advanced_regressor)
])

In [84]:
# Pipeline D = preproc2 + advanced model (random forest)
pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', advanced_regressor)
])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#pipelineA
param_grid_A = {
    'regressor__fit_intercept': [True, False]
}

# Create GridSearchCV object for Pipeline A
grid_search_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV for Pipeline A
grid_search_A.fit(X_train, y_train)

# Print the best parameters and score for Pipeline A
print("Best parameters for Pipeline A:", grid_search_A.best_params_)
print("Best RMSE for Pipeline A:", np.sqrt(-grid_search_A.best_score_))


85944.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
85944.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
85944.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
85944.05s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
85944.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
85944.07s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
85944.08s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
85944.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Best parameters for Pipeline A: {'regressor__fit_intercept': True}
Best RMSE for Pipeline A: 73.00899064224292


In [86]:
# pipelineB
param_grid_B = {
    'regressor__fit_intercept': [True, False]
}

grid_search_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search_B.fit(X_train, y_train)


print("Best parameters for Pipeline B:", grid_search_B.best_params_)
print("Best RMSE for Pipeline B:", np.sqrt(-grid_search_B.best_score_))

Best parameters for Pipeline B: {'regressor__fit_intercept': False}
Best RMSE for Pipeline B: 72.58129907088649


In [87]:
# Pipeline C (Random Forest)
param_grid_C = {
    'regressor__n_estimators': [50, 100, 150],
    'regressor__max_depth': [None, 10, 20]
}

grid_search_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search_C.fit(X_train, y_train)

print("Best parameters for Pipeline C:", grid_search_C.best_params_)
print("Best RMSE for Pipeline C:", np.sqrt(-grid_search_C.best_score_))

Best parameters for Pipeline C: {'regressor__max_depth': 10, 'regressor__n_estimators': 50}
Best RMSE for Pipeline C: 77.22017679875455


In [88]:
# Pipeline D
param_grid_D = {
    'regressor__n_estimators': [50, 100, 150],
    'regressor__max_depth': [None, 10, 20]
}

grid_search_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search_D.fit(X_train, y_train)

print("Best parameters for Pipeline D:", grid_search_D.best_params_)
print("Best RMSE for Pipeline D:", np.sqrt(-grid_search_D.best_score_))

Best parameters for Pipeline D: {'regressor__max_depth': 20, 'regressor__n_estimators': 150}
Best RMSE for Pipeline D: 77.19189807999051


# Evaluate

+ Which model has the best performance?

# Export
+ Save the best performing model to a pickle file.

In [89]:
#export
import pickle
with open('best_pipeline_B.pkl', 'wb') as file:
    pickle.dump(grid_search_B.best_estimator_, file)

print("Pipeline B has been saved to 'best_pipeline_B.pkl'")

Pipeline B has been saved to 'best_pipeline_B.pkl'


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [97]:
# import shap

# data_transform = pipeline_B.named_steps['preprocessing'].transform(X_test)
# explainer = shap.Explainer(
#     pipeline_B.named_steps['regressor'], 
#     data_transform,
#     feature_names=pipeline_B.named_steps['preprocessing'].get_feature_names_out()
# )



NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.